In [1]:
import sys
import os
import json

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
from Raydium.adapter.RaydiumAdapter import RaydiumAdapter
from SaucerSwap.adapter.SaucerSwapAdapter import SaucerSwapAdapter

In [2]:
import requests, json

CONFIG_PATH = "/root/Repositorios/LiquidityProvider/SaucerSwap/config/hedera.saucerswap.yaml"
adapter = SaucerSwapAdapter(CONFIG_PATH)

owner_evm = adapter.evm_address
helper_evm = adapter._get_whbar_helper_address()
token_evm  = adapter._whbar_helper_token_address()
owner_id   = adapter.account_id
token_id   = adapter._evm_to_hts(token_evm)    # esperado: 0.0.1456986
spender_id = adapter._evm_to_hts(helper_evm)   # esperado: 0.0.5808826

print("owner_evm:", owner_evm, "owner_id:", owner_id)
print("helper_evm:", helper_evm, "spender_id:", spender_id)
print("whbar_evm:", token_evm, "token_id:", token_id)

# 1) Allowance vía ERC-20 (referencial)
print("erc20.allowance(owner, helper):", adapter._erc20_allowance(token_evm, owner_evm, helper_evm))

# 2) Allowance vía Mirror Node (la fuente de verdad HTS)
base = "https://mainnet.mirrornode.hedera.com/api/v1"
url = f"{base}/accounts/{owner_id}/allowances/tokens?spender.id={spender_id}&token.id={token_id}"
r = requests.get(url, timeout=15); r.raise_for_status()
data = r.json()
print("mirror allowances:", json.dumps(data, indent=2))

# 3) Si no hay allowance, forzar approve HTS (SDK) y re-chequear en mirror
if not data.get("allowances"):
    res = adapter.approve_hts_execute(token_id=token_id, spender_contract_id=spender_id)
    print("approve_hts_execute:", res)
    r2 = requests.get(url, timeout=15); r2.raise_for_status()
    print("mirror allowances after approve:", json.dumps(r2.json(), indent=2))

owner_evm: 0x3845BbD7047E65A1336a42B647487f5dc4baD93A owner_id: 0.0.9637418
helper_evm: 0x000000000000000000000000000000000058a2ba spender_id: 0.0.5808826
whbar_evm: 0x0000000000000000000000000000000000163b5a token_id: 0.0.1456986
erc20.allowance(owner, helper): 0
mirror allowances: {
  "allowances": [],
  "links": {
    "next": null
  }
}
approve_hts_execute: {'executed': True, 'status': '<com.hedera.hashgraph.sdk.Status at 0x7af98a5d1770 jclass=com/hedera/hashgraph/sdk/Status jself=<LocalRef obj=0x6257efeb1c72 at 0x7afa113df430>>'}
mirror allowances after approve: {
  "allowances": [],
  "links": {
    "next": null
  }
}


In [3]:
CONFIG_PATH = "/root/Repositorios/LiquidityProvider/SaucerSwap/config/hedera.saucerswap.yaml"
adapter = SaucerSwapAdapter(CONFIG_PATH)

owner = adapter.evm_address
helper = adapter._get_whbar_helper_address()
whbar  = adapter._whbar_helper_token_address()  # token WHBAR desde el helper

print("Owner:", owner)
print("WhbarHelper:", helper)
print("WHBAR token:", whbar)

# Mostrar balance antes
bal_before = adapter._erc20_balance_of(whbar, owner)
print("Balance WHBAR before (raw):", bal_before, "human:", (bal_before or 0)/1e8)

# Barrer: approve si falta y unwrapWhbar(balance)
sweep = adapter._whbar_sweep_unwrap(owner)
print("Sweep result:", sweep)

# Balance después
bal_after = adapter._erc20_balance_of(whbar, owner)
print("Balance WHBAR after (raw):", bal_after, "human:", (bal_after or 0)/1e8)

Owner: 0x3845BbD7047E65A1336a42B647487f5dc4baD93A
WhbarHelper: 0x000000000000000000000000000000000058a2ba
WHBAR token: 0x0000000000000000000000000000000000163b5a
Balance WHBAR before (raw): 666374960 human: 6.6637496
Sweep result: {'executed': True, 'tx': {'txHash': '0x7374c9504b6d5908011437a0f16a269d808fe0135a2511025a2cfd573d04c2ea', 'receipt': {'blockHash': '0x358d013a176420fff5f7597d9d508985bdc4d08f478cbe9887560c55248082b1', 'blockNumber': '0x50bba2c', 'from': '0x3845bbd7047e65a1336a42b647487f5dc4bad93a', 'to': '0x000000000000000000000000000000000058a2ba', 'cumulativeGasUsed': '0xc54e6', 'gasUsed': '0xc54e6', 'contractAddress': '0x000000000000000000000000000000000058a2ba', 'logs': [{'address': '0x000000000000000000000000000000000058a2ba', 'blockHash': '0x358d013a176420fff5f7597d9d508985bdc4d08f478cbe9887560c55248082b1', 'blockNumber': '0x50bba2c', 'data': '0x0000000000000000000000000000000000000000000000000000000027b81330', 'logIndex': '0x0', 'removed': False, 'topics': ['0x5f2147fb